In [ ]:
import pandas as pd
import random

from mongo_aggregation_verbs import *

from lib import create_mongo_client_to_database_collection

collection_reference = create_mongo_client_to_database_collection('twitter', 'tweets')

In [ ]:
INSTAGRAM = '<a href="http://instagram.com" rel="nofollow">Instagram</a>'
source_is_instagram = { 'source' : INSTAGRAM }
source_is_not_instagram = { 'source' : {'$ne' : INSTAGRAM } } 

In [ ]:
(collection_reference.count_documents(source_is_instagram),
 collection_reference.count_documents(source_is_not_instagram))

## Tweet Locations

In [ ]:
not_null = { '$ne' : None }
nonnull_geo = {'geo' : not_null }
keep_geo_and_text = { 'geo' : 1, 'text' : 1 , '_id' : 0}

match_insta = {
    MATCH : source_is_instagram
}

match_not_insta = {
    MATCH : source_is_not_instagram
}

In [ ]:
cursor = collection_reference.aggregate([
    { MATCH : source_is_instagram },
    { MATCH : nonnull_geo},
    { COUNT : "geo"}
])
next(cursor)

In [ ]:
cursor = collection_reference.aggregate([
    { MATCH : source_is_not_instagram },
    { MATCH : nonnull_geo},
    { COUNT : "geo"}
])
next(cursor)

In [ ]:
def group_and_count(key):
    return { GROUP : {
                 "_id"   : key,
                 "count" : { "$sum" : 1 }
                }
           }

In [ ]:
def parse_geo_from_tweets(tweets):
    tweets = pd.DataFrame(tweets)
    geo = pd.DataFrame(list(tweets['_id'].values))
    geo['count'] = tweets['count']
    return geo

In [ ]:
cursor = collection_reference.aggregate([
    { MATCH : source_is_not_instagram },
    { MATCH : nonnull_geo},
    group_and_count('$geo'),
    { MATCH : { "count" : { "$gt" : 1 } } }, 
    { SORT  : { "count" : -1 } }
])
not_insta = parse_geo_from_tweets(list(cursor))
not_insta

In [ ]:
cursor = collection_reference.aggregate([
    { MATCH : source_is_instagram },
    { MATCH : nonnull_geo},
    group_and_count('$geo'),
    { MATCH : { "count" : { "$gt" : 1 } } }, 
    { SORT  : { "count" : -1 } }
])
insta = parse_geo_from_tweets(list(cursor))
insta

In [ ]:
import folium
starting_loc = [34.0689, -118.4452]
la_map = folium.Map(location=starting_loc, zoom_start=12)

In [ ]:
for loc, count in not_insta[['coordinates','count']].values:
    popup = folium.Popup(str(count), parse_html=True)
    folium.Marker(loc, popup=popup, icon=folium.Icon(color='red')).add_to(la_map)
for loc, count in insta[['coordinates','count']].values:
    popup = folium.Popup(str(count), parse_html=True)
    folium.Marker(loc, popup=popup, icon=folium.Icon(color='blue')).add_to(la_map)



In [ ]:
la_map

In [ ]:
def parse_geo_from_tweets(tweets):
    tweets = pd.DataFrame(tweets)
    geo = pd.DataFrame(list(tweets['_id'].values))
    geo['count'] = tweets['count']
    return geo

In [ ]:
cursor = collection_reference.aggregate([
    { MATCH : source_is_not_instagram },
    { MATCH : nonnull_geo},
    group_and_count('$user.id'),
    { MATCH : { "count" : { "$gt" : 1 } } }, 
    { SORT  : { "count" : -1 } },
    { LIMIT : 10 }
])
not_insta_top_users = pd.DataFrame(list(cursor))
not_insta_top_users

In [ ]:
cursor = collection_reference.aggregate([
    { MATCH : source_is_instagram },
    { MATCH : nonnull_geo},
    group_and_count('$user.id'),
    { MATCH : { "count" : { "$gt" : 1 } } }, 
    { SORT  : { "count" : -1 } },
    { LIMIT : 10 }
])
insta_top_users = pd.DataFrame(list(cursor))
insta_top_users

In [ ]:
not_insta_top_users_ids = not_insta_top_users._id.values
insta_top_users_ids = insta_top_users._id.values

In [ ]:
not_insta_top_users_ids_list = list(not_insta_top_users_ids)
not_insta_top_users_ids_list = [int(i) for i in not_insta_top_users_ids_list]
insta_top_users_ids_list = list(insta_top_users_ids)
insta_top_users_ids_list = [int(i) for i in insta_top_users_ids_list]

In [ ]:
def parse_geo_from_tweets(tweets):
    tweets = pd.DataFrame(tweets)
    tmp = pd.DataFrame(list(tweets['_id'].values))
    geo = pd.DataFrame(list(tmp['geo'].values))
    geo['user_id'] = tmp['user_id']
    geo['count'] = tweets['count']
    return geo

In [ ]:
cursor = collection_reference.aggregate([
    { MATCH : source_is_not_instagram },
    { MATCH : nonnull_geo},
    { PROJECT : { "user_id" : "$user.id", "geo" : 1, "text" : 1, "_id" :0 } },
    { MATCH : { "user_id" : { "$in" : not_insta_top_users_ids_list }}},
    group_and_count({"user_id" : "$user_id", "geo" : "$geo"}),
])

not_insta_top_user_geo = parse_geo_from_tweets(list(cursor))

In [ ]:
cursor = collection_reference.aggregate([
    { MATCH : source_is_instagram },
    { MATCH : nonnull_geo},
    { PROJECT : { "user_id" : "$user.id", "geo" : 1, "text" : 1, "_id" :0 } },
    { MATCH : { "user_id" : { "$in" : insta_top_users_ids_list }}},
    group_and_count({"user_id" : "$user_id", "geo" : "$geo"}),
])
insta_top_user_geo = parse_geo_from_tweets(list(cursor))

In [ ]:
not_insta_top_user_geo.head()

In [ ]:
insta_top_user_geo.head()

In [ ]:
colors = ['red','blue','green','purple','orange','darkred','lightred','beige','darkblue','darkgreen',]

In [ ]:
not_insta_top_user_geo_top_10 = not_insta_top_user_geo.sort_values('count').head(10)
insta_top_user_geo_top_10 = insta_top_user_geo.sort_values('count').head(10)

In [ ]:
insta_top_user_geo_top_10['color'] = colors
not_insta_top_user_geo_top_10['color'] = colors

In [ ]:
starting_loc = [34.0689, -118.4452]
la_map = folium.Map(location=starting_loc, zoom_start=12)

for loc, color, count in insta_top_user_geo_top_10[['coordinates','color','count']].values:
    popup = folium.Popup(str(count), parse_html=True)
    folium.Marker(loc, popup=popup, icon=folium.Icon(color=color)).add_to(la_map)


la_map

    760160463833313280 : 'red',
    30723561 : 'blue',
    613833206 : 'green',
    2267807461 : 'purple',
    4561143733 : 'orange',
    1455659006 : 'darkred',
    37016954 : 'lightred',
    19640448 : 'beige',
    843390093012353024 : 'darkblue',
    226456467 : 'darkgreen',

In [ ]:
starting_loc = [34.0689, -118.4452]
la_map = folium.Map(location=starting_loc, zoom_start=12)

for loc, color, count in not_insta_top_user_geo_top_10[['coordinates','color','count']].values:
    popup = folium.Popup(str(count), parse_html=True)
    folium.Marker(loc, popup=popup, icon=folium.Icon(color=color)).add_to(la_map)


la_map